In [26]:
import pandas as pd
import os
import datetime
import numpy as np
os.getcwd()

# Converted from seconds to minutes with 2 decimals

'/home/jian/Projects/Saatva/MMM/Input'

In [27]:
sales_data=pd.read_csv('/home/jian/Projects/Saatva/MMM/Input/sales_data.csv',dtype=str)
sales_data_saatva=sales_data[sales_data['brand']=='saatva']
sales_data_saatva['sales']=sales_data_saatva['sales'].astype(float).round(2)

sales_data_saatva['date']=sales_data_saatva['date_time'].apply(lambda x: datetime.datetime.strptime(x.split(" ")[0],"%Y-%m-%d").date())
sales_data_2018_Saatva=sales_data_saatva[sales_data_saatva['date'].apply(lambda x: x.year==2018)]
sales_data_2018_Saatva['date']=sales_data_2018_Saatva['date'].astype(str)
sales_data_2018_Saatva_sales=sales_data_2018_Saatva.groupby(['date'])['sales'].sum().to_frame().reset_index().rename(columns={"sales":"sales_net"})
sales_data_2018_Saatva_trans=sales_data_2018_Saatva.groupby(['date'])['sales'].count().to_frame().reset_index().rename(columns={"sales":"trans_net"})
sales_data_2018_Saatva=pd.merge(sales_data_2018_Saatva_sales,sales_data_2018_Saatva_trans,on="date")
sales_data_2018_Saatva.shape

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

(365, 3)

In [28]:
SM_data=pd.read_csv("/home/jian/Projects/Saatva/MMM/Input/all_sm_0409.csv",dtype=str)
del SM_data['Unnamed: 0']

for col in SM_data.columns.tolist()[1:]:
    SM_data[col]=SM_data[col].astype(float)
    if "Time on Page" in col:
        SM_data[col]=SM_data[col].apply(lambda x: np.round(x/60,2))
SM_data=SM_data[SM_data['date'].apply(lambda x: x.split("-")[0]=="2018")]
SM_data=pd.merge(SM_data,sales_data_2018_Saatva,on="date",how="inner")
SM_data.shape

median_types=SM_data.columns.tolist()
median_types=[x for x in median_types if "Sessions_" in x]
median_types=[x.split("_")[1] for x in median_types]
median_types.remove("NetTotal")


for col in median_types:
    SM_data['Avg Time On Page_'+col]=SM_data['Time on Page_'+col]/SM_data['Users_'+col]
    SM_data['Avg Time On Page_'+col]=SM_data['Avg Time On Page_'+col].fillna(0)
    
print(SM_data.shape)
sorted_columns_list_0=['date']+["Users_"+x for x in median_types]+["Time on Page_"+x for x in median_types]+\
                    ["Sessions_"+x for x in median_types]+["Transactions_"+x for x in median_types]+\
                    ["ga:goal11Completions_"+x for x in median_types]+["Avg Time On Page_"+x for x in median_types]
sorted_columns_list_1=sorted_columns_list_0+[x for x in SM_data.columns.tolist() if x not in sorted_columns_list_0]

print(len(sorted_columns_list_1))   
SM_data=SM_data[sorted_columns_list_1]
SM_data.head(2)

(365, 86)
86


,date,Users_Affiliates,Users_Bing Brand,Users_Bing Non-Brand,Users_Facebook,Users_GDN Non-Remarketing,Users_Google Brand,Users_Google Non-Brand,Users_Google shopping,Users_SEO,...,Avg Time On Page_direct,Avg Time On Page_retargeting,Avg Time On Page_Yahoo Non-Remarketing,Sessions_NetTotal,Time on Page_NetTotal,Transactions_NetTotal,Users_NetTotal,ga:goal11Completions_NetTotal,sales_net,trans_net
0,2018-01-01,2213.0,139.0,2659.0,882.0,6439.0,1013.0,3271.0,530.0,853.0,...,3.958967,0.617692,0.0,25687.0,52876.65,165.0,23803.0,605.0,330007.80,232
1,2018-01-02,1852.0,133.0,2926.0,838.0,5907.0,986.0,3355.0,489.0,875.0,...,4.251077,0.764586,0.0,25297.0,49339.65,140.0,23532.0,553.0,316132.57,227


In [29]:
[x for x in SM_data.columns.tolist() if "Users_" in x]

['Users_Affiliates',
 'Users_Bing Brand',
 'Users_Bing Non-Brand',
 'Users_Facebook',
 'Users_GDN Non-Remarketing',
 'Users_Google Brand',
 'Users_Google Non-Brand',
 'Users_Google shopping',
 'Users_SEO',
 'Users_Youtube',
 'Users_direct',
 'Users_retargeting',
 'Users_Yahoo Non-Remarketing',
 'Users_NetTotal']

In [30]:
df_SM_goals_sessions=SM_data[['date','ga:goal11Completions_NetTotal','Sessions_NetTotal','Sessions_direct','Sessions_SEO']]

df_SM_goals_sessions=df_SM_goals_sessions.rename(columns={"ga:goal11Completions_NetTotal":"SM_goal11Completions_NetTotal",
                                                         "Sessions_NetTotal":"Sessions_SM_total",
                                                         "Sessions_direct":"Sessions_SM_direct",
                                                         "Sessions_SEO":"Sessions_SM_SEO"})
df_SM_goals_sessions['Sessions_SM_Direct_and_SEO']=df_SM_goals_sessions['Sessions_SM_direct']+df_SM_goals_sessions['Sessions_SM_SEO']


In [31]:
SM_front_page_new_sessions=pd.read_excel("/home/jian/Projects/Saatva/MMM/Input/URLs/sm_all_urls_0417.xlsx",dtype=str,sheetname="Home Page")
sessions_cols=[x for x in SM_front_page_new_sessions.columns.tolist() if "www.saatvamattress.com" in x]
for col in sessions_cols:
    SM_front_page_new_sessions[col]=SM_front_page_new_sessions[col].astype(int)

SM_front_page_new_sessions['HomePage_0']=SM_front_page_new_sessions['www.saatvamattress.com'] # 0 pure the www.saatvamattress.com
SM_front_page_new_sessions['HomePage_1']=SM_front_page_new_sessions[sessions_cols].sum(axis=1) # 1 sum of home page links
SM_front_page_new_sessions=SM_front_page_new_sessions[['Date','HomePage_0','HomePage_1']].rename(columns={"Date":"date"})

In [32]:
SM_front_page_sessions=pd.read_csv("/home/jian/Projects/Saatva/MMM/Input/sm_front_page.csv",dtype=str)
SM_front_page_sessions.columns=['date','Sessions_SM_front_page_old'] # old means the mobile sessions are not included
SM_front_page_sessions['date']=SM_front_page_sessions['date'].apply(lambda x: str(datetime.date(int("20"+x.split("/")[2]),int(x.split("/")[0]),int(x.split("/")[1]))))
SM_front_page_sessions.head(2)
df_SM_goals_sessions=pd.merge(df_SM_goals_sessions,SM_front_page_sessions,on="date",how="outer")
df_SM_goals_sessions=pd.merge(df_SM_goals_sessions,SM_front_page_new_sessions,on="date",how="outer")
df_SM_goals_sessions.shape

(365, 9)

In [33]:
SC_data=pd.read_csv("/home/jian/Projects/Saatva/MMM/Input/all_sc_0326.csv",dtype=str)
del SC_data['Unnamed: 0']

for col in SC_data.columns.tolist()[1:]:
    SC_data[col]=SC_data[col].astype(float)
    if "Time on Page" in col:
        SC_data[col]=SC_data[col].apply(lambda x: np.round(x/60,2))
        
SC_data=SC_data[SC_data['date'].apply(lambda x: x.split("-")[0]=="2018")]
SC_data=pd.merge(SC_data,sales_data_2018_Saatva,on="date",how="inner")

SC_data=pd.merge(SC_data,df_SM_goals_sessions,how="inner",on="date")
print(SC_data.shape)

median_types=SC_data.columns.tolist()
median_types=[x for x in median_types if "Users_" in x]
median_types=[x.split("_")[1] for x in median_types]
median_types.remove("NetTotal")


for col in median_types:
    SC_data['Avg Time On Page_'+col]=SC_data['Time on Page_'+col]/SC_data['Users_'+col]
    SC_data['Avg Time On Page_'+col]=SC_data['Avg Time On Page_'+col].fillna(0)
    
print(SC_data.shape)
sorted_columns_list_0=['date']+["Users_"+x for x in median_types]+["Time on Page_"+x for x in median_types]+\
                    ["Sessions_"+x for x in median_types]+["Transactions_"+x for x in median_types]+\
                    ["ga:goal11Completions_"+x for x in median_types]+["Avg Time On Page_"+x for x in median_types]
sorted_columns_list_1=sorted_columns_list_0+[x for x in SC_data.columns.tolist() if x not in sorted_columns_list_0]

print(len(sorted_columns_list_1))   


(365, 76)
(365, 88)
88


In [34]:
SC_data['Cyber_Monday']=0
SC_data.loc[SC_data['date']=="2018-11-26",'Cyber_Monday']=1

SM_data['Cyber_Monday']=0
SM_data.loc[SM_data['date']=="2018-11-26",'Cyber_Monday']=1

In [35]:
SC_data.to_csv("/home/jian/Projects/Saatva/MMM/Input/Saatva_SC_2018_MMM_input_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
SM_data.to_csv("/home/jian/Projects/Saatva/MMM/Input/Saatva_SM_2018_MMM_input_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [36]:
SC_data['Sessions_SM_total'].dtype

dtype('float64')

# Replacing the outlier sessions

In [37]:
SC_data_replace=SC_data.copy()

replace_dates_str_list=['2018-01-12','2018-01-13','2018-02-10','2018-02-11','2018-02-12']
replace_dates_list=[datetime.datetime.strptime(x,"%Y-%m-%d").date() for x in replace_dates_str_list]
SC_data_replace['date']=SC_data_replace['date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
num_cols_list=SC_data_replace.columns.tolist()
num_cols_list.remove("date")



In [38]:
for col in num_cols_list:
    try:
        SC_data_replace[col]=SC_data_replace[col].astype(int)
    except:
        SC_data_replace[col]=SC_data_replace[col].astype(float)

In [39]:
for date_outlier in replace_dates_list:
    date_7_days_ago=date_outlier-datetime.timedelta(days=7)
    date_7_days_after=date_outlier+datetime.timedelta(days=7)
    
    for col in num_cols_list:
        
        col_values_7_days_ago=SC_data_replace.loc[SC_data_replace['date']==date_7_days_ago,col].values
        if len(col_values_7_days_ago)==1:
            col_values_7_days_ago=col_values_7_days_ago[0]
        
        col_values_7_days_after=SC_data_replace.loc[SC_data_replace['date']==date_7_days_after,col].values
        if len(col_values_7_days_after)==1:
            col_values_7_days_after=col_values_7_days_after[0]
        
        
        SC_data_replace.loc[SC_data_replace['date']==date_outlier,col]=(col_values_7_days_ago+col_values_7_days_after)/2

In [40]:
for col in median_types:
    SC_data_replace['Avg Time On Page_'+col]=SC_data_replace['Time on Page_'+col]/SC_data_replace['Users_'+col]
    SC_data_replace['Avg Time On Page_'+col]=SC_data_replace['Avg Time On Page_'+col].fillna(0)

In [41]:
SC_data_replace.to_csv("/home/jian/Projects/Saatva/MMM/Input/Saatva_SC_2018_MMM_input_replaced_sm_sessions_outliers_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [42]:
SC_data_replace.loc[SC_data_replace['date'].astype(str).isin(["2018-02-05","2018-02-12","2018-02-19"]),['date','HomePage_0']]

,date,HomePage_0
35,2018-02-05,19904.0
42,2018-02-12,26806.5
49,2018-02-19,33709.0
